# **Generative AI Use Case: Summarize Dialogue**

강의의 실습에 오신것을 환영합니다. 이 Lab에서는 생성AI를 이용해 담화 요약 task를 하게됩니다. 이곳에서는 입력 텍스트가 모델의 아웃풋에 미치는 영향을 알아보고, 원하는 task를 하기 위해 필요한 프롬프트 엔지니어링을 수행하게 됩니다. 우선 Zero shot, One shot, Few shot 인퍼런스를 비교하면서, 프롬프트 엔지니어링을 향한 첫 발을 디디고, 이후 그것들이 LLM의 출력을 어떻게 강화하는지 보게 됩니다.

**Table of Contents**
- 1 - Set up Kernel and Required Dependencies
- 2 - Summarize Dialogue without Prompt Engineering
- 3 - Summarize Dialogue with an Instruction Prompt
  - 3.1 - Zero Shot Inference with an Instruction Prompt
  - 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5
- 4 - Summarize Dialogue with One Shot and Few Shot Inference
  - 4.1 - One Shot Inference
  - 4.2 - Few Shot Inference
- 5 - Generative Configuration Parameters for Inference

## 1 - Set up Kernel and Required Dependencies

PyTorch와 Hugging Face transformers를 이용하기 위해 필요한 패키지를 설치합니다.

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 req

데이터셋, LLM, 토크나이저, 그리고 이를 위한 설정을 로드합니다. 각각의 요소가 무엇을 의미하는지는 몰라도 괜찮습니다. 뒤에서 언급됩니다.

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## 2 - Summarize Dialogue without Prompt Engineering

허깅페이스의 pre-trained된 FLAN-T5를 사용해 담와 요약문을 생성할 것입니다. 허깅페이스 `transformers`패키지에서 사용가능한 모델은 [여기](https://huggingface.co/docs/transformers/index)서 찾을 수 있습니다.

허깅페이스의 DialogSum 데이터셋과 함께 진행합니다. 이 데이터셋은 10000+개의 대화와 대응되는 손수 레이블링된 요약, 주제를 포함하고 있습니다.

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

베이스라인 요약과 함께 담화쌍을 프린트해봅니다.

In [5]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

AutoModelForSeq2SeqLM 클래스에 `.from_pretrained()` 메소드를 사용해 인스턴스를 제작하고, FLAN-T5 모델을 불러옵니다.

In [6]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

인코딩과 디코딩을 위해서, text를 토큰화된 형태로 변환해야합니다. **토큰화**란 텍스트를 LLM이 처리할 수 있는 작은 유닛으로 나누는 작업을 의미합니다.

FLAN-T5를 위한 토크나이저를 `AutoTokenizer.from_pretrained()` 메소드를 사용해 다운받습니다. `use_fast`파라미터는 fast tokenizer로 변경해줍니다. 이 단계에서, 디테일하게 더 볼 필요는 없지만, [여기](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoTokenizer)에서 토크나이저 파라미터들을 찾을 수 있습니다.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

간단한 문장을 인코딩/디코딩 하여 토크나이저를 테스트 해봅시다.

In [8]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


자, 이제 LLM이 아무런 프롬프트 엔지니어링 없이 얼마나 담화 요약을 잘 하는지 살펴보겠습니다. **프롬프트 엔지니어링**은 주어진 task에 대한 응답을 개선하기 위해 사람이 (입력)**프롬프트**를 바꾸는 행위를 이야기합니다.

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

모델이 수행한 추론이 어느정도 말은 되지만, 정확히 어떤 Task를 달성하려고 했는지 확실히 보이지는 않습니다. 그냥 다음에 올 말을 이어 만든 것 처럼 보입니다. 프롬프트 엔지니어링이 이를 해결할 수 있습니다.

## 3 - Summarize Dialogue with an Instruction Prompt

프롬프트 엔지니어링은 텍스트 생성을 위한 파운데이션 모델을 사용할 때, 중요한 개념입니다. [이곳](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) Amazon Science 블로그에서 프롬프트 엔지니어링을 위한 빠른 설명을 볼 수 있습니다.

### 3.1 - Zero Shot Inference with an Instruction Prompt

모델이 담화 요약 task를 수행하도록 지시하기 위해서는, 담화를 instruction prompt로 바꿀 수 있습니다. 이런 형태를 우리는 **zero-shot 인퍼런스**라고 부릅니다. [이곳](https://aws.amazon.com/ko/blogs/machine-learning/zero-shot-prompting-for-the-flan-t5-foundation-model-in-amazon-sagemaker-jumpstart/) AWS블로그에서 제로샷 러닝에 대한 빠른 설명과 이것이 LLM 모델에서 왜 중요한지 확인할 수 있습니다.

담화를 설명하는 지시문으로 감싼 후, 생성되는 텍스트가 어떻게 달라지는지 보도록 하겠습니다.

In [10]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to

훨씬 낫네요! 하지만 아직 모델이 담화를 관통하는 뉘앙스를 잡아내지 못하는 것을 볼 수 있습니다.

**Exercise:**
- `prompt` 텍스트를 가지고 인퍼런스 결과가 어떻게 변화하는지 실험해보세요. 문장을 아무런 설명 없이 끝내는 것과, `Summary:`로 끝나는 것이 결과에 차이를 주나요?
- `prompt` 시작부의 `Summarize the following conversation`을 재구성해보세요. 다른 것으로 바꾸고, 생성된 결과에 어떤 영향을 미치는지 알아보세요.

### 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5

살짝 다른 프롬프트를 사용해봅시다. FLAN-T5 모델은 [이곳](https://github.com/google-research/FLAN/tree/main/flan/v2)의 다양한 task를 위해 공개된 수많은 프롬프트를 가지고 있습니다. 아래 코드에서는, [미리 지어진 FLAN-T5 프롬프트](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py)를 사용하도록 하겠습니다.

In [11]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

--------------

FLAN-T5의 프롬프트가 어느정도 도움은 주었지만, 아직 대화의 뉘앙스를 포착하는 것을 어려워하는 것에 주목하세요. 이러한 문제들은 few shot inferencing을 통해 해결할 것들입니다.

## 4 - Summarize Dialogue with One Shot and Few Shot Inference

**One shot and few shot inference**는 실제로 완성하고 싶은 prompt의 전에, 수행하는 task에 대응되는 한개 또는 여러개의 prompt-response pairs을 LLM에 주어주는 관행입니다. "in-context learning"이라고 부르며, 모델을 특정한 task에 대해 이해하고있는 상태로 만들어줍니다. 더 많은 내용은 [HuggingFace 블로그](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api)에서 읽을 수 있습니다.

### 4.1 One Shot Inference

`example_indices_full` 리스트로 예시를 포함한 프롬프트를 만들고, 실제 수행할 `example_index_to_summerize`를 가장 뒤에 append해 최종 프롬프트를 생성하는 함수를 만들어봅시다. 3.2에서 사용한 프롬프트와 동일한 양식을 사용합니다.

In [12]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

One shot 인퍼런스를 위한 프롬프트를 만듭니다.

In [13]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

One shot 인퍼런스를 수행하기 위해, 모델에 이 프롬프트를 전달해봅시다.

In [14]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


## 4.2 - Few Shot Inference

프롬프트에 전체 대화-요약쌍을 2개 더 추가하여, few shot 인퍼런스를 해봅시다.

In [15]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

In [16]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


이 경우, One shot 인퍼런스에 비해 충분한 개선을 제공하지 못하였습니다. 그리고, 어떤 것이든 5~6개 이상의 shot은 통상적으로 그다지 도움이 되지 못합니다. 또한, 모델의 input-context 길이, 우리의 경우 512,를 넘지 않는지 확인해야 합니다. 이 길이를 넘는 context는 무시됩니다.

그러나, 적어도 한개의 full example(one shot)을 제공하는 것은, 모델에 더 많은 정보를 제공하고 질적으로 전반적인 요약 성능을 개선한다는 것을 알 수 있습니다.

**Exercise:**

few shot 인퍼런스로 실험해보세요.
- 다른 담화를 선택하세요. `example_indices_full`의 요소와 `example_index_to_summarize`를 변경하세요.
- shot의 개수를 바꾸어보세요. 512 context length 안에 들어오도록 유의하세요.

few shot 인퍼런스가 다른 예시에서 얼마나 잘 작동하나요?

## 5 - Generative Configuration Parameters for Inference

`generate()`의 설정 파라미터를 바꾸어 LLM의 결과가 어떻게 달라지는지 확인해보세요. 지금까지 선택한 유일한 토큰은 생성될 토큰의 최대 길이를 결정하는 `max_new_tokens=50`이었습니다. 사용가능한 전체 파라미터 리스트는 [Hugging Face Generation documentation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig)에서 찾을 수 있습니다.

설정 파라미터를 쉽게 구성하는 편리한 방법은 `GenerationConfig` Class를 사용하는 것입니다.

**Exercise:**

설정 파라미터를 바꾸고, 이들이 출력에 미치는 영향을 조사해보세요.

`do_sample = True`로 설정하면, 전체 Vocabulary에 걸친 확률 분포로부터 유도되는 next token에 영향을 미칠 다양한 디코딩 방법론을 활성화할 수 있습니다. 설정 시 temparature나 기타 파라미터(top_k나 top_p같은)를 바꿀 수 있습니다.

아래 셀의 줄을 uncomment한 후 코드를 재실행해본 후 결과를 분석해보세요.

In [17]:
generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



위 셀에서 선택한 파라미터에 대한 설명입니다.
- `max_new_tokens=10`은 출력의 길이를 너무 짧게 만들어, 요약된 문장이 잘렸습니다.
- `do_sample=True`와 Temparature을 바꾸는 것은 출력이 많은 flextibity를 가지게 하였습니다.

위에서 볼 수 있듯, 프롬프트 엔지니어링은 이 작어벵 큰 도움이 될 수 있지만 한계가 있었습니다. 다음은, 모델이 특정한 케이스에 대한 깊은 이해를 가질 수 있도록 fine-tune하는 방법을 알아보도록 하겠습니다.